In [1]:
import os
import torch
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document
from dotenv import load_dotenv

In [2]:
# .env 파일 로드
load_dotenv()

# OpenAI API 키를 환경 변수에서 로드
openai_api_key = os.getenv("OPENAI_API_KEY")

# Parquet 파일 경로 설정
file_path = 'merged_data.parquet'

# Parquet 파일을 pandas로 로드
df = pd.read_parquet(file_path)

# 각 행을 Document 객체로 변환
documents = []
for index, row in df.iterrows():
    content = f"""
    filename: {row['filename']}
    date: {row['date']}
    conference_number: {row['conference_number']}
    question_number: {row['question_number']}
    meeting_name: {row['meeting_name']}
    generation_number: {row['generation_number']}
    committee_name: {row['committee_name']}
    meeting_number: {row['meeting_number']}
    session_number: {row['session_number']}
    agenda: {row['agenda']}
    law: {row['law']}
    qna_type: {row['qna_type']}
    context: {row['context']}
    questioner_name: {row['questioner_name']}
    questioner_affiliation: {row['questioner_affiliation']}
    questioner_position: {row['questioner_position']}
    question_comment: {row['question_comment']}
    answerer_name: {row['answerer_name']}
    answerer_affiliation: {row['answerer_affiliation']}
    answerer_position: {row['answerer_position']}
    answer_comment: {row['answer_comment']}
    """
    documents.append(Document(page_content=content))

In [3]:
# 문서 분할 및 임베딩 생성
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

# CUDA 사용 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 임베딩 모델 설정 (GPU 사용)
embedding_model_name = 'sentence-transformers/all-MiniLM-L6-v2'
embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={'device': device}
)

# FAISS 인덱스 설정
index_path = 'faiss_index'

# VectorStore 생성 또는 로드
if os.path.exists(index_path):
    print("저장된 FAISS 인덱스를 로드합니다...")
    vectorstore = FAISS.load_local(
        index_path,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  # 여기서 True로 설정
    )
else:
    print("FAISS 인덱스를 생성합니다...")
    vectorstore = FAISS.from_documents(split_docs, embedding_model)
    vectorstore.save_local(index_path)

/tmp/ipykernel_205656/3851003835.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/home/elicer/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


저장된 FAISS 인덱스를 로드합니다...


In [4]:
# Retriever 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 6})

# 도구 함수 정의
def parquet_search_tool(input_text):
    docs = retriever.get_relevant_documents(input_text)
    if docs:
        summaries = [doc.page_content for doc in docs]
        return '\n\n'.join(summaries)
    else:
        return "해당하는 정보를 찾을 수 없습니다."

# 도구 정의
parquet_search_tool = Tool(
    name="ParquetSearch",
    func=parquet_search_tool,
    description=(
        "Use this tool to search for information about a specific person or issue from the parquet document. "
        "Provides a summary of fields such as conference number,date, meeting name, generation number, committee name, agenda, law, Q&A type, context, learning context, "
        "context summary, questioner, and question."
    )
)

# 도구 목록 생성
tools = [parquet_search_tool]

In [5]:
# LLM 정의
llm = ChatOpenAI(model_name="gpt-4", temperature=0, openai_api_key=openai_api_key)

# 메모리 설정
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 에이전트 초기화
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)

# 시스템 프롬프트 설정
agent.agent.llm_chain.prompt.messages[0].prompt.template = (
"Be sure to answer in Korean"
    "You are an AI assistant that helps users find information about individuals or issues from the National Assembly parquet document. "
    "When a user asks about a specific person or issue, you should use the JSONSearch tool to find meeting information where that person is mentioned. "
    "Provide a summarized response including fields such as conference number,date, meeting name, generation number, committee name, agenda, law, Q&A type, context, learning context, context summary, questioner, and question. "
    "If you cannot find the information in the JSON document, politely inform the user that the information is not available. "
    "Do not mention the use of tools; just provide the necessary information."
)

# 에이전트와 대화하는 함수 정의
def chat_with_agent(user_input):
    response = agent.run(input=user_input)
    return response

/tmp/ipykernel_205656/2021609686.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4", temperature=0, openai_api_key=openai_api_key)
/tmp/ipykernel_205656/2021609686.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
/tmp/ipykernel_205656/2021609686.py:8: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent

In [6]:
# 예시 질문
user_input = "김영삼 대통령이 언급된 회의에 대한 정보를 알려주세요. 반드시 안건 이름과 날짜, 어떤 회의였는지, 몇 대 국회였는지 알려주세요. 질문자와 답변자에 대한 정보와 어떤 안건에 대해 이야기가 오고 갔는지 대화문도 알려주세요."
response = chat_with_agent(user_input)
print("Assistant:", response)

/tmp/ipykernel_205656/2021609686.py:28: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(input=user_input)




> Entering new AgentExecutor chain...
```json
{
    "action": "ParquetSearch",
    "action_input": "김영삼"
}
```

/tmp/ipykernel_205656/1759276577.py:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(input_text)



Observation: answerer_name: 이희범
    answerer_affiliation: 
    answerer_position: 2018평창동계올림픽대회및동계패럴림픽대회조직위원장
    answer_comment: 예, 알고 있습니다.   국민안전처를 중심으로 해서 저희 조직위원회하고 통합안전망체제를 지금 하고 있고요. 몇 차례 회의도 지금 하고 있습니다.

answerer_name: 김현숙
    answerer_affiliation: 새만금개발청
    answerer_position: 청장
    answer_comment: 아니요, 지금 말씀을……

answerer_name: 안종범
    answerer_affiliation: 대통령비서실
    answerer_position: 정책조정수석비서관
    answer_comment: 제가 말씀 좀 드리겠습니다.

answerer_name: 홍남기
    answerer_affiliation: 부총리겸기획재정부
    answerer_position: 장관
    answer_comment: 위원님 지금 말씀 주신 대로 탄소중립은 저희가 선택할 수 있는 사안이 아니고 필수로 가야 되고요. 이것을……

answerer_name: 홍남기
    answerer_affiliation: 기획재정부
    answerer_position: 부총리겸기획재정부장관
    answer_comment: 예.

answerer_name: 홍남기
    answerer_affiliation: 기획재정부
    answerer_position: 부총리겸기획재정부장관
    answer_comment: 예.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "김영삼 대통령에 대한 언급이 포함된 회의에서 다음과 같은 정보를 찾을 수 있었습니다. \n\n1. 이희범이라는 사람이 2018평창동계올림픽대회및동계패럴림픽대회조직위원장으로서 참여하였

In [7]:
# 예시 질문
user_input = "안건 날짜와 어떤 몇대 국회에서 열린지 어떤 회의였는지 알려줘야지."
response = chat_with_agent(user_input)
print("Assistant:", response)



> Entering new AgentExecutor chain...
```json
{
    "action": "ParquetSearch",
    "action_input": "안건 날짜, 국회 대수, 회의 이름"
}
```
Observation: answerer_name: 이희범
    answerer_affiliation: 
    answerer_position: 2018평창동계올림픽대회및동계패럴림픽대회조직위원장
    answer_comment: 예, 알고 있습니다.   국민안전처를 중심으로 해서 저희 조직위원회하고 통합안전망체제를 지금 하고 있고요. 몇 차례 회의도 지금 하고 있습니다.

answerer_name: 홍남기
    answerer_affiliation: 기획재정부장
    answerer_position: 부총리겸기획재정부장관
    answer_comment: 위원님 말씀 경청하겠습니다. 유념하겠고요. 저로서는 4/4분기까지 정부가 할 수 있는 모든 수단을 동원해서 일단 저희가 약속드렸던 그런 내용들이 달성될 수 있도록 최선을 다하겠습니다.

answerer_name: 조기열
    answerer_affiliation: 
    answerer_position: 전문위원
    answer_comment: 보고드리겠습니다.  국회법 79조의2(의안에 대한 비용추계자료 등의 제출)에 나와 있습니다. 그래서 의원 발의 법률안을 제출할 때는 비용추계서를 첨부하거나 예산정책처에 대한 비용추계요구서 둘 중 하나를 제출해야 되고요, 비용추계서는 해당 위원회의 심사 전까지 제출되어야 하는 것으로 되어 있습니다.

answerer_name: 김광묵
    answerer_affiliation: 
    answerer_position: 수석전문위원
    answer_comment: 그렇지요. 국가공무원법 63조에 ‘품위가 손상되는 행위를 하여서는 아니 된다’라고 두 번째 줄에 되어 있기 때문에 추상적이긴 하지만 선례가 있는 문구라고 보시면 